# Практика: Предобработка, токенизация и извлечение признаков

В этой тетради мы пройдём:
- Различные методы токенизации
- Нормализацию и очистку текста
- Стемминг и лемматизацию
- Извлечение признаков: BoW, TF-IDF, BM25, N-граммы
- Работу с OOV словами

Установите необходимые библиотеки:
```
pip install nltk pymorphy2 scikit-learn rank-bm25 unicodedata
```

## 1. Методы токенизации: сравнение подходов

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize
import pymorphy2
from collections import Counter

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

print("=" * 70)
print("1. СРАВНЕНИЕ МЕТОДОВ ТОКЕНИЗАЦИИ")
print("=" * 70)

# Сложный текст с разными случаями
complex_text = """Mr. Smith e-mailed john@example.com at 5:00 p.m. regarding the project.
It's a very important matter! What's the status?"""

print(f"\nТекст:\n{complex_text}\n")

# 1. Простая токенизация по пробелам
simple = complex_text.split()
print(f"1. split() — {len(simple)} токенов:")
print(f"   {simple}")
print("   ⚠️ Проблема: пунктуация и апострофы не разделены\n")

# 2. NLTK word_tokenize
nltk_tokens = word_tokenize(complex_text)
print(f"2. NLTK word_tokenize() — {len(nltk_tokens)} токенов:")
print(f"   {nltk_tokens}")
print("   ✓ Хорошо разделяет пунктуацию\n")

# 3. Regex токенизация
regex_tokens = re.findall(r"\b\w+\b|[\w.,!?;'-]", complex_text.lower())
print(f"3. Regex (word + punct) — {len(regex_tokens)} токенов:")
print(f"   {regex_tokens}")
print("   ✓ Контролируем, что считать токеном\n")

1. СРАВНЕНИЕ МЕТОДОВ ТОКЕНИЗАЦИИ

Текст:
Mr. Smith e-mailed john@example.com at 5:00 p.m. regarding the project.
It's a very important matter! What's the status?

1. split() — 18 токенов:
   ['Mr.', 'Smith', 'e-mailed', 'john@example.com', 'at', '5:00', 'p.m.', 'regarding', 'the', 'project.', "It's", 'a', 'very', 'important', 'matter!', "What's", 'the', 'status?']
   ⚠️ Проблема: пунктуация и апострофы не разделены

2. NLTK word_tokenize() — 25 токенов:
   ['Mr.', 'Smith', 'e-mailed', 'john', '@', 'example.com', 'at', '5:00', 'p.m.', 'regarding', 'the', 'project', '.', 'It', "'s", 'a', 'very', 'important', 'matter', '!', 'What', "'s", 'the', 'status', '?']
   ✓ Хорошо разделяет пунктуацию

3. Regex (word + punct) — 35 токенов:
   ['mr', '.', 'smith', 'e', '-', 'mailed', 'john', 'example', '.', 'com', 'at', '5', '00', 'p', '.', 'm', '.', 'regarding', 'the', 'project', '.', 'it', "'", 's', 'a', 'very', 'important', 'matter', '!', 'what', "'", 's', 'the', 'status', '?']
   ✓ Контролируем,

In [3]:
print("\n" + "=" * 70)
print("2. ПОДСЛОВНАЯ ТОКЕНИЗАЦИЯ: BPE (Byte Pair Encoding)")
print("=" * 70)

class SimpleBPE:
    """Упрощённая реализация BPE токенизатора"""
    
    def __init__(self, num_merges=100):
        self.num_merges = num_merges
        self.word_freqs = {}
        self.bpe_ranks = {}
        self.vocab = {}
    
    def _get_stats(self, vocab):
        """Подсчитаем частоту каждой пары токенов"""
        pairs = {}
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                pairs[pair] = pairs.get(pair, 0) + freq
        return pairs
    
    def _merge_vocab(self, pair, vocab):
        """Объединяем самую частую пару во всех словах"""
        new_vocab = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        
        for word in vocab:
            new_word = word.replace(bigram, replacement)
            new_vocab[new_word] = vocab[word]
        
        return new_vocab
    
    def train(self, texts):
        """Обучить BPE на корпусе текстов"""
        # Считаем частоту слов
        for text in texts:
            for word in text.split():
                # Инициализируем словом, разделённым на символы
                word_lower = word.lower()
                char_word = ' '.join(list(word_lower)) + ' </w>'
                self.word_freqs[char_word] = self.word_freqs.get(char_word, 0) + 1
        
        vocab = {word: freq for word, freq in self.word_freqs.items()}
        
        # Выполняем меджи
        for i in range(self.num_merges):
            pairs = self._get_stats(vocab)
            if not pairs:
                break
            
            best_pair = max(pairs, key=pairs.get)
            self.bpe_ranks[best_pair] = i
            vocab = self._merge_vocab(best_pair, vocab)
        
        # Сохраняем финальный словарь
        self.vocab = set(vocab.keys())
    
    def tokenize(self, word):
        """Применяем BPE к слову"""
        word_lower = word.lower()
        word = ' '.join(list(word_lower)) + ' </w>'
        
        # Применяем обученные мержи
        for pair, rank in sorted(self.bpe_ranks.items(), key=lambda x: x[1]):
            bigram = ' '.join(pair)
            replacement = ''.join(pair)
            word = word.replace(bigram, replacement)
        
        return word.split()

# Пример использования
texts = [
    "the quick brown fox jumps",
    "the lazy dog",
    "the quick brown fox",
]

bpe = SimpleBPE(num_merges=50)
bpe.train(texts)

print(f"\nОригинальное слово 'quick': {list('quick')}")
print(f"BPE токены: {bpe.tokenize('quick')}")

print(f"\nОригинальное слово 'independence': {list('independence')}")
print(f"BPE токены: {bpe.tokenize('independence')}")
print("\n✓ BPE разбивает редкие слова на более частые подслова")


2. ПОДСЛОВНАЯ ТОКЕНИЗАЦИЯ: BPE (Byte Pair Encoding)

Оригинальное слово 'quick': ['q', 'u', 'i', 'c', 'k']
BPE токены: ['quick</w>']

Оригинальное слово 'independence': ['i', 'n', 'd', 'e', 'p', 'e', 'n', 'd', 'e', 'n', 'c', 'e']
BPE токены: ['i', 'n', 'd', 'e', 'p', 'e', 'n', 'd', 'e', 'n', 'c', 'e', '</w>']

✓ BPE разбивает редкие слова на более частые подслова


## 2. Нормализация текста

In [4]:
import unicodedata
import html

print("=" * 70)
print("НОРМАЛИЗАЦИЯ ТЕКСТА")
print("=" * 70)

# Примеры проблемных текстов
texts_to_normalize = [
    "HELLO World! это ТЕКСТ.",
    "café naïve résumé",  # диакритические знаки
    "<p>HTML &amp; encoded text</p>",  # HTML
    "   Лишние    пробелы   везде  ",  # пробелы
    "привет ПРИВЕТ ПрИвЕт",  # регистр
]

class TextNormalizer:
    """Полная нормализация текста"""
    
    @staticmethod
    def normalize(text, lowercase=True, remove_accents=True, 
                  remove_html=True, clean_whitespace=True):
        """
        Полная нормализация:
        - Преобразование регистра
        - Удаление диакритических знаков
        - HTML декодирование
        - Очистка пробелов
        """
        # 1. Удаление HTML
        if remove_html:
            text = html.unescape(text)
            text = re.sub(r'<[^>]+>', '', text)
        
        # 2. Приведение к нижнему регистру
        if lowercase:
            text = text.lower()
        
        # 3. Удаление диакритических знаков (accents)
        if remove_accents:
            text = unicodedata.normalize('NFD', text)
            text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
        
        # 4. Очистка пробелов
        if clean_whitespace:
            text = re.sub(r'\s+', ' ', text)  # множественные пробелы → один
            text = text.strip()  # начало/конец
        
        return text

normalizer = TextNormalizer()

print("\nПримеры нормализации:")
for text in texts_to_normalize:
    normalized = normalizer.normalize(text)
    print(f"\nОригинал:    '{text}'")
    print(f"Нормализовано: '{normalized}'")

НОРМАЛИЗАЦИЯ ТЕКСТА

Примеры нормализации:

Оригинал:    'HELLO World! это ТЕКСТ.'
Нормализовано: 'hello world! это текст.'

Оригинал:    'café naïve résumé'
Нормализовано: 'cafe naive resume'

Оригинал:    '<p>HTML &amp; encoded text</p>'
Нормализовано: 'html & encoded text'

Оригинал:    '   Лишние    пробелы   везде  '
Нормализовано: 'лишние пробелы везде'

Оригинал:    'привет ПРИВЕТ ПрИвЕт'
Нормализовано: 'привет привет привет'


## 3. Стемминг vs Лемматизация

In [5]:
from nltk.stem.snowball import SnowballStemmer

print("=" * 70)
print("СТЕММИНГ VS ЛЕММАТИЗАЦИЯ")
print("=" * 70)

# Примеры словоформ
words_ru = ['бежали', 'бегущий', 'бежал', 'бегун', 'работают', 'рабочий', 'работой']
words_en = ['running', 'runs', 'ran', 'runner', 'working', 'works', 'worked']

# Инициализация инструментов
stemmer_en = SnowballStemmer('english')
morph = pymorphy2.MorphAnalyzer()

print("\nАНГЛИЙСКИЙ ЯЗЫК:")
print("-" * 70)
print(f"{'Слово':<20} {'Стемминг':<20} {'Результат':<20}")
print("-" * 70)

for word in words_en:
    stem = stemmer_en.stem(word)
    print(f"{word:<20} {stem:<20}")

print("\nРУССКИЙ ЯЗЫК:")
print("-" * 70)
print(f"{'Слово':<20} {'Лемматизация':<20}")
print("-" * 70)

for word in words_ru:
    lemma = morph.parse(word)[0].normal_form
    print(f"{word:<20} {lemma:<20}")

print("\n" + "=" * 70)
print("СРАВНЕНИЕ:")
print("=" * 70)
print("\nСТЕММИНГ (Stemming):")
print("  + Быстрый (просто алгоритм отсечения)")
print("  - Может дать не реальное слово (напр. 'run' + 'ing' → 'runn')")
print("  - Не учитывает контекст и часть речи")
print("\nЛЕММАТИЗАЦИЯ (Lemmatization):")
print("  + Возвращает реальное, словарное слово")
print("  + Учитывает морфологию")
print("  - Медленнее (требует словаря и анализа)")

СТЕММИНГ VS ЛЕММАТИЗАЦИЯ

АНГЛИЙСКИЙ ЯЗЫК:
----------------------------------------------------------------------
Слово                Стемминг             Результат           
----------------------------------------------------------------------
running              run                 
runs                 run                 
ran                  ran                 
runner               runner              
working              work                
works                work                
worked               work                

РУССКИЙ ЯЗЫК:
----------------------------------------------------------------------
Слово                Лемматизация        
----------------------------------------------------------------------
бежали               бежать              
бегущий              бежать              
бежал                бежать              
бегун                бегун               
работают             работать            
рабочий              рабочий             
работой

## 4. Обработка OOV (Out-of-Vocabulary) слов

In [6]:
print("=" * 70)
print("ОБРАБОТКА OOV (Out-of-Vocabulary) СЛОВ")
print("=" * 70)

class SimpleVocabulary:
    """Словарь с обработкой OOV слов"""
    
    def __init__(self, min_freq=2, max_vocab_size=1000):
        self.min_freq = min_freq
        self.max_vocab_size = max_vocab_size
        self.word2id = {}  # слово → ID
        self.id2word = {}  # ID → слово
        self.word_freq = Counter()
        
        # Специальные токены
        self.pad_token = '<PAD>'
        self.unk_token = '<UNK>'
        self.sos_token = '<SOS>'
        self.eos_token = '<EOS>'
        
        self._add_special_tokens()
    
    def _add_special_tokens(self):
        special_tokens = [self.pad_token, self.unk_token, self.sos_token, self.eos_token]
        for token in special_tokens:
            idx = len(self.word2id)
            self.word2id[token] = idx
            self.id2word[idx] = token
    
    def build(self, texts):
        """Построить словарь из корпуса"""
        # Считаем частоту слов
        for text in texts:
            tokens = text.lower().split()
            self.word_freq.update(tokens)
        
        # Добавляем слова, которые встречаются >= min_freq
        for word, freq in self.word_freq.most_common(self.max_vocab_size):
            if freq >= self.min_freq:
                idx = len(self.word2id)
                self.word2id[word] = idx
                self.id2word[idx] = word
    
    def encode(self, text):
        """Преобразовать текст в IDs, неизвестные слова → UNK"""
        tokens = text.lower().split()
        ids = []
        for token in tokens:
            if token in self.word2id:
                ids.append(self.word2id[token])
            else:
                ids.append(self.word2id[self.unk_token])
        return ids
    
    def decode(self, ids):
        """Преобразовать IDs обратно в текст"""
        tokens = [self.id2word.get(id, self.unk_token) for id in ids]
        return ' '.join(tokens)
    
    def size(self):
        return len(self.word2id)

# Пример использования
train_texts = [
    'the quick brown fox jumps',
    'the lazy dog sleeps',
    'quick fox jumps over dog',
]

vocab = SimpleVocabulary(min_freq=1, max_vocab_size=100)
vocab.build(train_texts)

print(f"\nРазмер словаря: {vocab.size()} токенов")
print(f"\nСловарь: {vocab.word2id}")

# Тестирование OOV
test_text = "the UNKNOWN_WORD fox ANOTHER_UNKNOWN"
encoded = vocab.encode(test_text)
decoded = vocab.decode(encoded)

print(f"\nОригинальный текст: {test_text}")
print(f"Закодирован: {encoded}")
print(f"Декодирован: {decoded}")
print("\nЗамечание: неизвестные слова (UNKNOWN_WORD, ANOTHER_UNKNOWN) → <UNK>")

ОБРАБОТКА OOV (Out-of-Vocabulary) СЛОВ

Размер словаря: 13 токенов

Словарь: {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3, 'the': 4, 'quick': 5, 'fox': 6, 'jumps': 7, 'dog': 8, 'brown': 9, 'lazy': 10, 'sleeps': 11, 'over': 12}

Оригинальный текст: the UNKNOWN_WORD fox ANOTHER_UNKNOWN
Закодирован: [4, 1, 6, 1]
Декодирован: the <UNK> fox <UNK>

Замечание: неизвестные слова (UNKNOWN_WORD, ANOTHER_UNKNOWN) → <UNK>


## 5. Извлечение признаков: BoW, TF-IDF, N-граммы

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import math

print("=" * 70)
print("ИЗВЛЕЧЕНИЕ ПРИЗНАКОВ: Bag of Words и TF-IDF")
print("=" * 70)

# Маленький корпус
documents = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are pets",
]

# 1. Bag of Words (Count Vectorizer)
print("\n1. BAG OF WORDS (BoW)")
print("-" * 70)

bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(documents)

print(f"\nМатрица BoW формы: {X_bow.shape}")
print(f"Словарь: {bow_vectorizer.get_feature_names_out()}")

print("\nМатрица (документы × слова):")
print(X_bow.toarray())
print("\n✓ BoW показывает частоту каждого слова в документе")

# 2. TF-IDF
print("\n" + "-" * 70)
print("2. TF-IDF (Term Frequency - Inverse Document Frequency)")
print("-" * 70)

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(documents)

print(f"\nМатрица TF-IDF формы: {X_tfidf.shape}")
print("\nМатрица (документы × слова):")
print(X_tfidf.toarray())

print("\nФормулы:")
print("  TF(t, d) = count(t in d) / total_words(d)")
print("  IDF(t) = log(total_docs / docs_with_t)")
print("  TF-IDF(t, d) = TF(t, d) × IDF(t)")

# Анализ: слово 'the' в первом документе
word_idx = list(bow_vectorizer.get_feature_names_out()).index('the')
tf_value = X_bow[0, word_idx] / X_bow[0].sum()
idf_value = math.log(3 / 3)  # все 3 документа имеют 'the'

print("\nПример: слово 'the' в документе 1")
print(f"  TF = {X_bow[0, word_idx]}/6 = {tf_value:.3f}")
print(f"  IDF = log(3/3) = {idf_value:.3f}")
print(f"  TF-IDF = {X_tfidf[0, word_idx]:.3f}")
print("  💡 Низкое TF-IDF, потому что слово встречается везде (низкая IDF)")

# Слово 'mat' в первом документе (редкое)
word_idx = list(bow_vectorizer.get_feature_names_out()).index('mat')
tf_value = X_bow[0, word_idx] / X_bow[0].sum()
idf_value = math.log(3 / 1)  # только 1 документ имеет 'mat'

print("\nПример: слово 'mat' в документе 1")
print(f"  TF = {X_bow[0, word_idx]}/6 = {tf_value:.3f}")
print(f"  IDF = log(3/1) = {idf_value:.3f}")
print(f"  TF-IDF = {X_tfidf[0, word_idx]:.3f}")
print("  💡 Высокое TF-IDF, потому что слово редкое (высокая IDF)")

ИЗВЛЕЧЕНИЕ ПРИЗНАКОВ: Bag of Words и TF-IDF

1. BAG OF WORDS (BoW)
----------------------------------------------------------------------

Матрица BoW формы: (3, 12)
Словарь: ['and' 'are' 'cat' 'cats' 'dog' 'dogs' 'log' 'mat' 'on' 'pets' 'sat' 'the']

Матрица (документы × слова):
[[0 0 1 0 0 0 0 1 1 0 1 2]
 [0 0 0 0 1 0 1 0 1 0 1 2]
 [1 1 0 1 0 1 0 0 0 1 0 0]]

✓ BoW показывает частоту каждого слова в документе

----------------------------------------------------------------------
2. TF-IDF (Term Frequency - Inverse Document Frequency)
----------------------------------------------------------------------

Матрица TF-IDF формы: (3, 12)

Матрица (документы × слова):
[[0.         0.         0.42755362 0.         0.         0.
  0.         0.42755362 0.32516555 0.         0.32516555 0.6503311 ]
 [0.         0.         0.         0.         0.42755362 0.
  0.42755362 0.         0.32516555 0.         0.32516555 0.6503311 ]
 [0.4472136  0.4472136  0.         0.4472136  0.         0.4472136


In [8]:
print("\n" + "=" * 70)
print("3. N-ГРАММЫ (N-grams)")
print("=" * 70)

text = "the quick brown fox jumps over the lazy dog"

def get_ngrams(text, n):
    """Извлечь n-граммы из текста"""
    tokens = text.split()
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngram = ' '.join(tokens[i:i+n])
        ngrams.append(ngram)
    return ngrams

print(f"\nТекст: '{text}'")

# Юниграммы (1-граммы)
unigrams = get_ngrams(text, 1)
print(f"\n1-граммы: {unigrams}")

# Биграммы (2-граммы)
bigrams = get_ngrams(text, 2)
print(f"\n2-граммы: {bigrams}")

# Триграммы (3-граммы)
trigrams = get_ngrams(text, 3)
print(f"\n3-граммы: {trigrams}")

# N-граммы через sklearn
print("\n" + "-" * 70)
print("N-граммы через TfidfVectorizer:")
print("-" * 70)

documents = [
    "the quick brown fox",
    "the lazy brown dog",
]

# Только биграммы
bigram_vectorizer = TfidfVectorizer(ngram_range=(2, 2))
X_bigrams = bigram_vectorizer.fit_transform(documents)

print("\nБиграммы (ngram_range=(2,2)):")
print(f"  {list(bigram_vectorizer.get_feature_names_out())}")

# Комбинация унигрмм и биграмм
combined_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_combined = combined_vectorizer.fit_transform(documents)

print("\nУниграммы + Биграммы (ngram_range=(1,2)):")
print(f"  {list(combined_vectorizer.get_feature_names_out())}")

print(f"\nМатрица формы: {X_combined.shape}")
print(f"  (документы: {X_combined.shape[0]}, признаки: {X_combined.shape[1]})")


3. N-ГРАММЫ (N-grams)

Текст: 'the quick brown fox jumps over the lazy dog'

1-граммы: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

2-граммы: ['the quick', 'quick brown', 'brown fox', 'fox jumps', 'jumps over', 'over the', 'the lazy', 'lazy dog']

3-граммы: ['the quick brown', 'quick brown fox', 'brown fox jumps', 'fox jumps over', 'jumps over the', 'over the lazy', 'the lazy dog']

----------------------------------------------------------------------
N-граммы через TfidfVectorizer:
----------------------------------------------------------------------

Биграммы (ngram_range=(2,2)):
  ['brown dog', 'brown fox', 'lazy brown', 'quick brown', 'the lazy', 'the quick']

Униграммы + Биграммы (ngram_range=(1,2)):
  ['brown', 'brown dog', 'brown fox', 'dog', 'fox', 'lazy', 'lazy brown', 'quick', 'quick brown', 'the', 'the lazy', 'the quick']

Матрица формы: (2, 12)
  (документы: 2, признаки: 12)


## 6. BM25 для информационного поиска

In [10]:
from rank_bm25 import BM25Okapi

print("=" * 70)
print("BM25 ДЛЯ ИНФОРМАЦИОННОГО ПОИСКА")
print("=" * 70)

# Корпус документов
documents = [
    "machine learning is a subset of artificial intelligence",
    "deep learning uses neural networks with multiple layers",
    "natural language processing analyzes text data",
    "computer vision processes visual information",
    "neural networks are inspired by biological neurons",
]

# Токенизируем
tokenized_docs = [doc.lower().split() for doc in documents]

# Инициализируем BM25
bm25 = BM25Okapi(tokenized_docs)

# Поисковые запросы
queries = [
    "neural networks learning",
    "text processing language",
    "artificial intelligence",
]

print("\nКорпус документов:")
for i, doc in enumerate(documents, 1):
    print(f"  {i}. {doc}")

print("\n" + "-" * 70)
print("РЕЗУЛЬТАТЫ ПОИСКА BM25:")
print("-" * 70)

for query in queries:
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    
    print(f"\nЗапрос: '{query}'")
    print("  Документ   | Оценка BM25 | Результат")
    print("  " + "-" * 40)
    
    for i, score in enumerate(scores, 1):
        bar = '█' * int(score * 10)
        print(f"  Doc {i}     | {score:10.4f} | {bar}")

print("\nФормула BM25:")
print("  Учитывает:")
print("    1. Частоту слова в документе (TF)")
print("    2. Обратную частоту документа (IDF)")
print("    3. Длину документа (нормализация)")
print("    4. Насыщение (diminishing returns для частых слов)")
print("  ✓ Лучше чем TF-IDF для информационного поиска")

BM25 ДЛЯ ИНФОРМАЦИОННОГО ПОИСКА

Корпус документов:
  1. machine learning is a subset of artificial intelligence
  2. deep learning uses neural networks with multiple layers
  3. natural language processing analyzes text data
  4. computer vision processes visual information
  5. neural networks are inspired by biological neurons

----------------------------------------------------------------------
РЕЗУЛЬТАТЫ ПОИСКА BM25:
----------------------------------------------------------------------

Запрос: 'neural networks learning'
  Документ   | Оценка BM25 | Результат
  ----------------------------------------
  Doc 1     |     0.3117 | ███
  Doc 2     |     0.9352 | █████████
  Doc 3     |     0.0000 | 
  Doc 4     |     0.0000 | 
  Doc 5     |     0.6642 | ██████

Запрос: 'text processing language'
  Документ   | Оценка BM25 | Результат
  ----------------------------------------
  Doc 1     |     0.0000 | 
  Doc 2     |     0.0000 | 
  Doc 3     |     3.4801 | ████████████████████████

## 7. Практические задачи

In [11]:
print("=" * 70)
print("ПРАКТИЧЕСКИЕ ЗАДАЧИ И ВЫЗОВЫ")
print("=" * 70)

# ЗАДАЧА 1: Сравнение векторизаций на реальном примере
print("\n" + "=" * 70)
print("ЗАДАЧА 1: Выбор правильной векторизации")
print("=" * 70)

# Корпус новостей (пример)
news_docs = [
    "политика президент министр парламент",
    "спорт футбол олимпиада чемпионат",
    "технология компьютер программа интернет",
    "политика выборы голос закон",
    "спорт команда игрок стадион",
]

# Сравниваем разные подходы
approaches = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer(),
    'Bigramы': TfidfVectorizer(ngram_range=(2, 2)),
}

print("\nСравнение размеров матриц:")
for name, vectorizer in approaches.items():
    X = vectorizer.fit_transform(news_docs)
    print(f"  {name:15} → {X.shape} (документы × признаки)")

# Анализ: какие признаки важны?
tfidf_vec = TfidfVectorizer()
X_tfidf = tfidf_vec.fit_transform(news_docs)

print("\nТоп признаки для каждого документа:")
feature_names = tfidf_vec.get_feature_names_out()
for doc_idx in range(len(news_docs)):
    top_indices = X_tfidf[doc_idx].toarray()[0].argsort()[-3:][::-1]
    top_features = [feature_names[i] for i in top_indices]
    top_scores = X_tfidf[doc_idx].toarray()[0][top_indices]
    print(f"  Doc {doc_idx + 1}: {', '.join([f'{f}({s:.2f})' for f, s in zip(top_features, top_scores)])}")

ПРАКТИЧЕСКИЕ ЗАДАЧИ И ВЫЗОВЫ

ЗАДАЧА 1: Выбор правильной векторизации

Сравнение размеров матриц:
  BoW             → (5, 18) (документы × признаки)
  TF-IDF          → (5, 18) (документы × признаки)
  Bigramы         → (5, 15) (документы × признаки)

Топ признаки для каждого документа:
  Doc 1: президент(0.52), парламент(0.52), министр(0.52)
  Doc 2: чемпионат(0.52), футбол(0.52), олимпиада(0.52)
  Doc 3: технология(0.50), программа(0.50), интернет(0.50)
  Doc 4: выборы(0.52), голос(0.52), закон(0.52)
  Doc 5: стадион(0.52), игрок(0.52), команда(0.52)


In [12]:
# ЗАДАЧА 2: Обработка несбалансированных классов
print("\n" + "=" * 70)
print("ЗАДАЧА 2: Несбалансированные классы")
print("=" * 70)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Симулируем несбалансированный набор
positive_docs = [
    "отличный продукт очень доволен",
    "супер качество рекомендую всем",
    "лучший выбор когда либо",
] * 5  # много примеров

negative_docs = [
    "ужасный продукт не работает",
    "разочарован совсем",
]  # мало примеров

all_docs = positive_docs + negative_docs
labels = [1] * len(positive_docs) + [0] * len(negative_docs)

print("\nДисбаланс классов:")
print(f"  Класс 1 (позитив): {sum(labels)} примеров ({sum(labels)/len(labels)*100:.1f}%)")
print(f"  Класс 0 (негатив): {len(labels)-sum(labels)} примеров ({(1-sum(labels)/len(labels))*100:.1f}%)")
print(f"  Соотношение: {sum(labels)/(len(labels)-sum(labels)):.1f}:1")

# Векторизация
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_docs)

# Обучение
clf = LogisticRegression(class_weight='balanced')  # Важно! balanced weights
clf.fit(X, labels)

# Предсказание
y_pred = clf.predict(X)

print("\nПредсказания на обучающей выборке:")
print(classification_report(labels, y_pred, target_names=['Негатив', 'Позитив']))

print("\nПримечание: class_weight='balanced' дает больший вес редкому классу")


ЗАДАЧА 2: Несбалансированные классы

Дисбаланс классов:
  Класс 1 (позитив): 15 примеров (88.2%)
  Класс 0 (негатив): 2 примеров (11.8%)
  Соотношение: 7.5:1

Предсказания на обучающей выборке:
              precision    recall  f1-score   support

     Негатив       1.00      1.00      1.00         2
     Позитив       1.00      1.00      1.00        15

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17


Примечание: class_weight='balanced' дает больший вес редкому классу


In [13]:
# ЗАДАЧА 3: PMI для выделения коллокаций
print("\n" + "=" * 70)
print("ЗАДАЧА 3: PMI и NPMI для коллокаций")
print("=" * 70)

def compute_pmi_npmi(tokens, window_size=2):
    """
    PMI (Pointwise Mutual Information) = log(P(w1, w2) / (P(w1) * P(w2)))
    NPMI (Normalized PMI) = PMI / -log(P(w1, w2))
    """
    from itertools import combinations
    from collections import defaultdict
    
    vocab_freq = Counter(tokens)  # P(w)
    pair_freq = defaultdict(int)  # P(w1, w2)
    
    # Считаем совместные частоты (окно слов)
    for i in range(len(tokens) - window_size + 1):
        window = tokens[i:i+window_size]
        for pair in combinations(sorted(set(window)), 2):
            pair_freq[pair] += 1
    
    total = sum(vocab_freq.values())
    pmi_scores = {}
    
    for (w1, w2), count in pair_freq.items():
        p_pair = count / total
        p_w1 = vocab_freq[w1] / total
        p_w2 = vocab_freq[w2] / total
        
        pmi = math.log(p_pair / (p_w1 * p_w2))
        npmi = pmi / -math.log(p_pair) if p_pair > 0 else 0
        
        pmi_scores[(w1, w2)] = (pmi, npmi, count)
    
    return pmi_scores

# Примеры текстов
text = """machine learning is powerful. machine learning systems use neural networks.
deep learning is neural networks. neural networks learn patterns.
machine learning and deep learning are related."""

tokens = text.lower().split()
pmi_scores = compute_pmi_npmi(tokens, window_size=3)

# Сортируем по PMI
sorted_pmi = sorted(pmi_scores.items(), key=lambda x: x[1][0], reverse=True)

print("\nТоп 10 коллокаций по PMI:")
print(f"{'Коллокация':<30} {'PMI':>10} {'NPMI':>10} {'Частота':>8}")
print("-" * 70)

for (w1, w2), (pmi, npmi, count) in sorted_pmi[:10]:
    print(f"{w1 + ' + ' + w2:<30} {pmi:10.4f} {npmi:10.4f} {count:8}")

print("\nВысокий PMI → слова часто встречаются вместе (коллокация)")
print("Низкий/отрицательный PMI → слова избегают друг друга (антиассоциация)")


ЗАДАЧА 3: PMI и NPMI для коллокаций

Топ 10 коллокаций по PMI:
Коллокация                            PMI       NPMI  Частота
----------------------------------------------------------------------
systems + use                      3.9512     1.5405        2
learn + networks                   3.9512     1.5405        2
learn + patterns.                  3.9512     1.5405        2
is + powerful.                     3.2581     1.2702        2
networks + patterns.               3.2581     1.0000        1
and + deep                         3.2581     1.2702        2
are + related.                     3.2581     1.0000        1
networks. + neural                 3.0758     1.8656        5
machine + powerful.                2.8526     1.1122        2
neural + use                       2.8526     1.1122        2

Высокий PMI → слова часто встречаются вместе (коллокация)
Низкий/отрицательный PMI → слова избегают друг друга (антиассоциация)
